In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [24]:
df = pd.read_csv(r"C:\Users\C J HARINI\Downloads\df_iemocap_with_features.csv")

In [25]:
df.head()

,start_time,end_time,wav_file,emotion,val,act,dom,MFCC1,MFCC2,MFCC3,...,Chroma11,Chroma12,SpectralContrast1,SpectralContrast2,SpectralContrast3,SpectralContrast4,SpectralContrast5,SpectralContrast6,SpectralContrast7,ZeroCrossingRate
0,8.0341,13.50,Ses05F_script01_1_F000,sad,2.5,2.5,2.5,-464.898987,119.969124,-7.643576,...,0.421615,0.375193,17.053972,16.371297,16.881244,17.185019,17.721546,17.477850,20.656050,0.078382
1,15.2259,19.10,Ses05F_script01_1_F001,xxx,2.0,3.0,3.0,-435.348572,117.497269,-10.906404,...,0.380136,0.421472,17.265016,16.791738,17.651642,17.205775,17.052922,17.942451,21.527245,0.094663
2,20.5700,22.10,Ses05F_script01_1_F002,xxx,2.5,2.0,2.5,-498.951324,135.751755,-14.035595,...,0.631194,0.421012,15.005613,14.258981,16.267633,17.036546,17.999499,17.367494,17.658056,0.060699
3,28.9800,32.13,Ses05F_script01_1_F003,xxx,2.5,3.0,2.5,-459.131958,127.234604,-10.658073,...,0.432782,0.388997,15.970424,17.969763,17.118443,17.177244,17.301360,18.076253,19.646218,0.064700
4,38.1100,39.88,Ses05F_script01_1_F004,sad,2.5,2.5,3.0,-472.654846,118.265526,-8.783584,...,0.453893,0.584195,15.275234,16.910328,16.387280,16.713556,17.352526,18.122145,19.914245,0.084891


In [26]:
mfcc_features = df[['MFCC1', 'MFCC2', 'MFCC3', 'MFCC4', 'MFCC5', 'MFCC6', 'MFCC7', 
                    'MFCC8', 'MFCC9', 'MFCC10', 'MFCC11', 'MFCC12', 'MFCC13','Chroma11', 'Chroma12', 'SpectralContrast1', 'SpectralContrast2', 'ZeroCrossingRate' ]].values
labels = df['emotion'].values

In [27]:
# Standardize the MFCC features
scaler = StandardScaler()
mfcc_features_scaled = scaler.fit_transform(mfcc_features)

In [28]:
# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [29]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(mfcc_features_scaled, labels_encoded, test_size=0.2, random_state=42)


In [30]:
# Build the improved DNN model with regularization and batch normalization
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(len(np.unique(y_train)), activation='softmax'))

In [31]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {test_acc}')

Epoch 1/50
55/55 [==============================] - 2s 7ms/step - loss: 2.4581 - accuracy: 0.1872 - val_loss: 2.0277 - val_accuracy: 0.3295
Epoch 2/50
55/55 [==============================] - 0s 3ms/step - loss: 2.1590 - accuracy: 0.2264 - val_loss: 1.8907 - val_accuracy: 0.3318
Epoch 3/50
55/55 [==============================] - 0s 4ms/step - loss: 1.9865 - accuracy: 0.2949 - val_loss: 1.7884 - val_accuracy: 0.3525
Epoch 4/50
55/55 [==============================] - 0s 4ms/step - loss: 1.8979 - accuracy: 0.3093 - val_loss: 1.7134 - val_accuracy: 0.3641
Epoch 5/50
55/55 [==============================] - 0s 5ms/step - loss: 1.8324 - accuracy: 0.3220 - val_loss: 1.6668 - val_accuracy: 0.3525
Epoch 6/50
55/55 [==============================] - 0s 5ms/step - loss: 1.7835 - accuracy: 0.3306 - val_loss: 1.6324 - val_accuracy: 0.3594
Epoch 7/50
55/55 [==============================] - 0s 4ms/step - loss: 1.7076 - accuracy: 0.3589 - val_loss: 1.6141 - val_accuracy: 0.3641
Epoch 8/50
55/55 [==